# Python to SQL, and back again
In this codealong we will show you how to create a relational database from your pandas DataFrames.
> **To run this notebook you will need to work locally and not on colab.**

---
## 1.&nbsp; Import libraries 💾
If you haven't already installed sqlalchemy, you will need to. Uncomment the code below, install, and then recomment the code - you only need to install it once.

In [ ]:
# install if needed
# !pip install sqlalchemy
# !pip install pymysql

In [ ]:
import pandas as pd

---
## 2.&nbsp; Relational Databases 📂

Creating DataFrames in python and pandas often results in tables with repeated information, as shown in the example below.
<br>

| Director | Movie | YearReleased |
| --- | --- | --- |
| Ridley Scott | Blade Runner | 1982 |
| Hayao Miyazaki | My Neighbor Totoro | 1988 |
| Ridley Scott | Thelma & Louise | 1991 |
| Hayao Miyazaki | Spirited Away | 2001 |
| Jennifer Kent | The Babadook | 2014 |
<br>

This can be problematic for relational databases, which are designed to store data efficiently and avoid redundancy. To address this issue, we will separate the director and movie information into two tables: directors and movies. This approach eliminates duplicate data, ensuring data integrity and optimising storage.
<br>

| DirectorId | DirectorName |
| --- | --- |
| 1 | Ridley Scott |
| 2 | Hayao Miyazaki |
| 3 | Jennifer Kent |
<br>

| MovieId | MovieTitle | YearReleased | DirectorId |
|---|---|---|---|
| 1 | Blade Runner | 1982 | 1 |
| 2 | My Neighbor Totoro | 1988 | 2 |
| 3 | Thelma & Louise | 1991 | 1 |
| 4 | Spirited Away | 2001 | 2 |
| 5 | The Babadook | 2014 | 3 |

---
## 3.&nbsp; Creating the directors table with python 🐍
Let's start by creating the original DataFrame, including the repeated data.

In [ ]:
data = {
    "Director": ["Ridley Scott", "Hayao Miyazaki", "Ridley Scott", "Hayao Miyazaki", "Jennifer Kent"],
    "Movie": ["Blade Runner", "My Neighbor Totoro", "Thelma & Louise", "Spirited Away", "The Babadook"],
    "YearReleased": [1982, 1988, 1991, 2001, 2014]
}

non_relational_df = pd.DataFrame(data)

non_relational_df

Now, let's select only the authors without any duplicates.

In [ ]:
directors_unique = non_relational_df["Director"].unique()

directors_df = pd.DataFrame({"DirectorName": directors_unique})

directors_df

Fantastic! This DataFrame will be the foundation of our directors table.

---
## 4.&nbsp; Creating the matching directors table with SQL 💻

Ok, now we're ready to store this DataFrame in SQL. Before we can send the information in SQL, we need to make a table that has the same columns and data types to recieve the data. While we are creating a table for directors, we can also create the movies table too.

Open MySQL Workbench, open a local connection, and open a new file. Then copy and paste the code from below.

```sql
-- Drop the database if it already exists
DROP DATABASE IF EXISTS SqlWorkshop ;

-- Create the database
CREATE DATABASE SqlWorkshop;

-- Use the database
USE SqlWorkshop;

-- Create the 'Directors' table
CREATE TABLE Directors (
    DirectorId INT AUTO_INCREMENT, -- Automatically generated ID
    DirectorName VARCHAR(255) NOT NULL, -- Name of the director
    PRIMARY KEY (DirectorId) -- Primary key to uniquely identify each director
);

-- Create the 'Movies' table
CREATE TABLE Movies (
    MovieId INT AUTO_INCREMENT, -- Automatically generated ID
    MovieTitle VARCHAR(255) NOT NULL, -- Title of the movie
    YearReleased INT, -- Year the movie was released
    DirectorId INT, -- ID of the director who directed the movie
    PRIMARY KEY (MovieId), -- Primary key to uniquely identify each movie
    FOREIGN KEY (DirectorId) REFERENCES Directors(DirectorId) -- Foreign key to connect each movie to its director
);
```

---
## 5.&nbsp; Sending the information from this notebook to sql 📠
To establish a connection with the SQL database, we need to provide the notebook with the necessary information, which we do using the connection string below. You will need to modify only the password variable, which should match the password you set during MySQL Workbench installation.

In [ ]:
schema = "SqlWorkshop"
host = "127.0.0.1"
user = "root"
password = "YOUR_PASSWORD"
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

To send information to our sql databse we use the pandas method [`.to_sql()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html). The argument `if_exists="append"` says that we don't want to overwrite any existing data, but add on to what is already there.

In [ ]:
directors_df.to_sql('Directors',
                    if_exists = 'append',
                    con = connection_string,
                    index = False)

Now, have a look at the table `Directors` in MySQL Workbench, you should see that the names of the directors have appeared.

---
## 6.&nbsp; Retrieving information from sql to this notebook 📥
It's not only possible to send information to a SQL database, but also retrieve it too. Using `.read_sql()` in combination with the `connection_string` we can access the required data.

In [ ]:
directors_from_sql = pd.read_sql("Directors",
                                 con = connection_string)
directors_from_sql

Using this same method, we can also perform SQL queries to only bring back certain sections of information instead of the whole DataFrame.

In [ ]:
pd.read_sql("""
            SELECT DISTINCT DirectorName
            FROM Directors
            """,
            con = connection_string)

---
## 7.&nbsp; Preparing and sending the movies table 📚
By extracting the directors table from our SQL database, we gain access to the unique identifier `DirectorId` assigned to each director. These `DirectorId`'s serve as pointers to their corresponding director records, allowing us to seamlessly link the `DirectorId`'s in the books table to their respective director in the directors table, thereby completing the movies table.

In [ ]:
movies_df = non_relational_df.merge(directors_from_sql,
                                    left_on = "Director",
                                    right_on = "DirectorName",
                                    how = "left")

movies_df

In [ ]:
movies_df = movies_df.drop(columns=["Director", "DirectorName"])
movies_df = movies_df.rename(columns={"Movie": "MovieTitle"})

movies_df

In [ ]:
movies_df.to_sql('Movies',
                 if_exists = 'append',
                 con = connection_string,
                 index=False)

In [ ]:
movies_from_sql = pd.read_sql("Movies",
                              con = connection_string)
movies_from_sql

That's it! We created our data in python and sent it to a SQL database.

---
## 8.&nbsp; Enhancing SQL Control with engine.connect() 🎛️
`.read_sql()` and `.to_sql()` are powerful tools for interacting with SQL databases, but they come with limitations. `.read_sql()` is designed primarily for running `SELECT` queries and loading the results into a DataFrame, while `.to_sql()` is used for inserting or replacing entire DataFrame contents into a table.

However, these methods do not support executing more complex SQL operations like `UPDATE`, `DELETE`, or transactional queries. Additionally, `.to_sql()` can overwrite data if not used cautiously and is not optimised for updating specific rows.

For more granular control over SQL operations, including updates, deletions, and transactional integrity, the `engine.connect()` method from SQLAlchemy is necessary. This approach allows you to execute raw SQL queries directly, manage transactions, and ensure precise control over how data is modified in the database.

Here's an example of how to update the data of a single cell:

In [ ]:
from sqlalchemy import create_engine, text

# Create an engine to connect to the database
engine = create_engine(connection_string)

# Define the update query
update_query = """
    UPDATE Movies
    SET MovieTitle = "I can change anything!"
    WHERE MovieId = 3;
"""

# Execute the update query with an explicit commit
with engine.connect() as connection:
    transaction = connection.begin()
    try:
        connection.execute(text(update_query))
        transaction.commit()
    except:
        transaction.rollback()
        raise

Let's see that change.

In [ ]:
pd.read_sql("""
            SELECT *
            FROM Movies
            """,
            con = connection_string)

Now let's change that back to the correct data.

In [ ]:
from sqlalchemy import create_engine, text

# Create an engine to connect to the database
engine = create_engine(connection_string)

# Define the update query
update_query = """
    UPDATE Movies
    SET MovieTitle = "Thelma & Louise"
    WHERE MovieId = 3;
"""

# Execute the update query with an explicit commit
with engine.connect() as connection:
    transaction = connection.begin()
    try:
        connection.execute(text(update_query))
        transaction.commit()
    except:
        transaction.rollback()
        raise

And, just double check that change took effect.

In [ ]:
pd.read_sql("""
            SELECT *
            FROM Movies
            """,
            con = connection_string)